In [1]:
import pandas as pd
import numpy as np

In [2]:
def logprior(b0, b1, b2, b3, sigma2):
    return -np.log(sigma2)

def logistic(t, h, w, b0, b1, b2, b3):
    #x[0]: t
    #x[1]: h
    #x[2]: w
    theta = b0 + x[0]*b1 + x[1]*b2 + x[2]*b3
    return 0.5/(1+np.exp(theta)) + 1

def loglikelihood(b0, b1, b2, b3, sigma2, x, y):
    n = len(x)
    return -n*np.log(sigma2) - 0.5*((y-logistic(x))**2).sum()/sigma2

def logpost(b0, b1, b2, b3, sigma2, x, y):
    return logprior(b0, b1, b2, b3, sigma2) + loglikelihood(b0, b1, b2, b3, sigma2, x, y)